# Predicting User Churn in Digital Music Services

Notebook to document data exploration and development of ML algorithm to identify at risk customers in digital music services.

### Data Definition

From Exploratory Data Analysis (EDA): 
#### Useful:
- *location*: location of user, seems to append each new state (location, state)
- *gender*: user gender (M/F/None)

- *page*: what page the user is on during event (pages)
- *level*: subscription level check uniqueness (free or paid)
- *auth*: authenication (logged in/out)
- *length*: time spent on page, max 50 mins on NextSong (if song paused??)

- *registration*: unknown (registration unixtime)
- *ts*: timestamp of event in ms (event unixtime)

- *userId*: unique (userId val)
- *sessionId*: unique sessionId per user?
- *itemInSession*: lcounter for the number of items in a single session (item listened to in session)


#### Not Useful:
- *firstName*: users first name (not important, remove)
- *lastName*: users lastname
- *artist*: song artist
- *song*: songname
- *userAgent*: device/browser (not important for us, remove)
- *method*: API PUT/GET http request (not important for us, remove)
- *status*: http status

# Apache Spark on IBM Watson Setup

### Imports

In [3]:
# imports
#import ibmos2spark

# pyspark sql
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import from_unixtime, udf, col, when, isnan, desc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import functions as F

# pyspark ml
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# python
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

### setup

In [2]:
# # config
# # @hidden_cell
# credentials = {
#     'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
#     'service_id': 'iam-ServiceId-147e1161-7da9-41fe-ac00-c144730def00',
#     'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
#     'api_key': 'kAtvjdC8VIYYUmU3gDaOYIK2fCvP3nkjYYlDiNuu4gw6'
# }

# configuration_name = 'os_76774389dfa04fb5acbb1640b3e11704_configs'
# cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [5]:
# Build Spark session
spark = SparkSession.builder.appName("user_churn").getOrCreate()

# Read in data from IBM Cloud
# data_df = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-fnqu5byx41gcai'))

data_df = spark.read.json("../data/01_raw/medium-sparkify-event-data.json")

# Exploratory Data Analysis

In [6]:
data_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Data Setup for ML Algorithm

In [ ]:
ml_df_prep=user_log_valid

## Create Features

### Onehot Encode Categorical Variables

In [ ]:
## https://stackoverflow.com/questions/32277576/how-to-handle-categorical-features-with-spark-ml
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

indexer = StringIndexer(inputCol="level", outputCol="levelIndex")
inputs = [indexer.getOutputCol()]
encoder = OneHotEncoderEstimator(inputCols=inputs, outputCols=["levelVec"])

pipeline = Pipeline(stages=[indexer, encoder])
ml_df_prep = pipeline.fit(ml_df_prep).transform(ml_df_prep)

### Create Features Vector

In [ ]:
# this vector is created in prep for ml
assembler = VectorAssembler(inputCols=["sessionId",
                                       "itemInSession",
                                       "hours_since_registration",
                                       "levelVec"],
                            outputCol="features",
                           handleInvalid="skip")
ml_df_prep = assembler.transform(ml_df_prep)

In [ ]:
# apply scaler
scaler = Normalizer(inputCol="features", outputCol="ScaledFeatures")
ml_df_prep = scaler.transform(ml_df_prep)

In [ ]:
ml_df_prep.head(1)

In [ ]:
ml_df = ml_df_prep.select("label","features")
ml_df.head()

# Train ML Model

In [ ]:
# train test split for ML validation
train, test =  ml_df.randomSplit([0.8, 0.2], seed=42)  # more equal fit to combat overfitting
train.head(50)

In [ ]:
train.filter(train['label']==1).count()

In [ ]:
train.filter(train['label']==0).count()

In [ ]:
train.filter(train['label']==1).count()/train.filter(train['label']==0).count()

In [ ]:
# estimators
lr = LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

### Baseline

baseline binary Logisitc Regression Model

In [ ]:
lrmodel = lr.fit(train)

In [ ]:
lr_results = lrmodel.transform(test) 

In [ ]:
lr_results.filter(lr_results["prediction"]==1).head(10)

In [ ]:
lrmodel.summary.accuracy

In [ ]:
lrmodel.summary.fMeasureByLabel()

In [ ]:
lrmodel.summary.precisionByLabel

### Optimised

In [ ]:
# pipeline, just running it on classifier no transformations
pipeline = Pipeline(stages=[lr])

In [ ]:
# set up param grid to iterate over
paramGrid = ParamGridBuilder() \
.addGrid(lr.regParam, [0.0, 0.1]) \
.build()

In [ ]:
# set up crossvalidator to tune parameters and optimize
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                         numFolds=2)

In [ ]:
cvModel = crossval.fit(train)  # train model

In [ ]:
results = cvModel.transform(test)  # apply model on test data

In [ ]:
cvModel.avgMetrics  # look at model scoring metrics

In [ ]:
results.count()  # how many events in total labels

In [ ]:
print(results.filter(results.label == results.prediction).count())  # check how many were predicted correctly

In [ ]:
results.filter(results.label == results.prediction).count()/results.count()   # hwow many correct

In [ ]:
results.filter(results["prediction"]==1).head(5)

In [ ]:
results.filter(results["label"]==1).head(50)